In [0]:
# DevSecOps Lab - Enable Lakebase Persistence
# After creating your Lakebase database in the GUI, fill in these details:

dbutils.widgets.text("lakebase_host", "", "1) Lakebase Host (e.g., workspace.cloud.databricks.com)")
dbutils.widgets.text("lakebase_database", "", "2) Database Name")
dbutils.widgets.dropdown("enable_chat_history", "true", ["true", "false"], "3) Enable Chat History")

lakebase_host = dbutils.widgets.get("lakebase_host")
lakebase_database = dbutils.widgets.get("lakebase_database")
enable_chat_history = dbutils.widgets.get("enable_chat_history")

print(f"✓ Lakebase Host: {lakebase_host}")
print(f"✓ Database: {lakebase_database}")
print(f"✓ Chat History: {enable_chat_history}")

In [0]:
import os

# Get current user dynamically
current_user = spark.sql("SELECT current_user()").collect()[0][0]

# Construct dynamic path to user's deployed app
app_path = f"/Workspace/Users/{current_user}/DevSecOps_Labs/chatbot-app"
env_file = f"{app_path}/.env.local"

# Build environment configuration
env_content = f"""# Lakebase Configuration (Auto-generated from notebook)
PGHOST={lakebase_host}
PGDATABASE={lakebase_database}
PGPORT=5432
PGSSLMODE=require

# Feature Flags
CHAT_HISTORY_ENABLED={enable_chat_history}

# Databricks Auth (uses your current session)
DATABRICKS_HOST=https://{lakebase_host}
DATABRICKS_SERVING_ENDPOINT=your-agent-endpoint
"""

# Write configuration file
with open(env_file, 'w') as f:
    f.write(env_content)

print(f"✓ Configuration written to {env_file}")
print(f"✓ App path: {app_path}")

In [0]:
# Create the database schema and tables
import subprocess
import os

os.chdir(f"{app_path}/packages/db")

print("Creating ai_chatbot schema and tables...")
result = subprocess.run(
    ["npx", "drizzle-kit", "push"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✓ Database schema created successfully!")
    print("\nTables created:")
    print("  - ai_chatbot.Chat")
    print("  - ai_chatbot.Message")
    print("  - ai_chatbot.User")
else:
    print("✗ Migration failed:")
    print(result.stderr)

In [0]:
# Verify the database connection and schema
from databricks import sql
import os

connection = sql.connect(
    server_hostname=lakebase_host,
    http_path=f"/sql/1.0/warehouses/{lakebase_database}",  # Adjust if needed
    access_token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
)

cursor = connection.cursor()

# Check if tables exist
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'ai_chatbot'
""")

tables = cursor.fetchall()
print("✓ Database tables found:")
for table in tables:
    print(f"  - ai_chatbot.{table[0]}")

cursor.close()
connection.close()

print("\n✓ Lakebase setup complete!")
print("\nNext steps:")
print("1. Restart your chatbot app")
print("2. Chat history will now be persisted to Lakebase")